## TL;DR

Join: `pd.merge()`

- Type
  - **One-to-one**: just like column-wise concatenation
    - Order of entries NOT necessarily maintained
    - In general discards the index
  - **Many-to-one**: one of the two key columns contains duplicate entries
    - Preserve those duplicate entries as appropriate
  - **Many-to-many**: key column in both the left and right array contains duplicates
  
  
- Specification of the merge key
  - Rules:
    - If joining columns on columns, the `DataFrame` indexes *will be ignored*.
    - If joining indexes on indexes or indexes on a column or columns, the index will be passed on.
  - `on`: Explicitly specify the name of the key column
    - Works only if both the left and right `DataFrame`s have the specified column name.
  - `left_on` and `right_on`: Column or index level names to join on in the **left** and **right** DataFrame.
  - `left_index` and `right_index`: merge on index
    - Mix indices and columns:
      - `left_index` + `right_on`
      - `left_on` + `right_index`
      
- Set Arithmetic
  - Inner join: 
    - `how=inner`
    - Result contains intersection of the two sets of inputs
  - Outer join: 
    -  `how=outer`
    - Returns a join over the union of the input columns, and fills in all missing values with NAs
  - Left join and right join:  return joins over the left entries and right entries, respectively.
    - *Keep the left/right entries standstill and merge the another*


- Overlapping column: Use `suffixes` keyword to specify column suffix







In [65]:
import pandas as pd
import numpy as np

## Relational Algebra

A formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.

Strength: it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset. 

## Join

The `pd.merge()` function implements a number of types of joins: 

-  *one-to-one*
- *many-to-one*
- *many-to-many* joins. 

All three types of joins are accessed via an identical call to the `pd.merge()` interface; the type of join performed depends on the form of the input data. 

### One-to-one joins

In many ways very similar to the column-wise concatenation 

In [66]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [67]:
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [68]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


- The `pd.merge()` function recognizes that each `DataFrame` has an "employee" column, and automatically joins using this column as a key.  

- The result of the merge is a new `DataFrame` that combines the information from the two inputs.

- Notice: the order of entries in each column is not necessarily maintained

- Note: the merge in general discards the index, except in the special case of merges by index 

### Many-to-one joins

Many-to-one joins are joins in which **one of the two key columns contains duplicate entries**. For the many-to-one case, the resulting `DataFrame` will preserve those duplicate entries as appropriate. 

In [69]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [70]:
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [71]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


The resulting `DataFrame` has an aditional column with the "supervisor" information, where the information is repeated in one or more locations (row with `Engineering`) as required by the inputs.

### Many-to-many joins

If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge. 

In [72]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [73]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [74]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


## Specification of the Merge Key

Default behavior of `pd.merge()`: it looks for one or more matching column names between the two inputs, and uses this as the key. 

However, often the column names will not match so nicely, and `pd.merge()` provides a variety of options for handling this.

The join is done on columns or indexes:

- If joining columns on columns, the DataFrame indexes *will be ignored*. 

- Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on.


### The `on` keyword

Explicitly specify the name of the key column using the `on` keyword, which takes a column name or a list of column names.

Note: it works only if both the left and right `DataFrame`s have the specified column name.

In [75]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [76]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [77]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [78]:
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [79]:
pd.merge(df1, df4, on='employee')

KeyError: 'employee'

### The `left_on` and `right_on` keywords

Merge two datasets with different column names

- `left_on`: Column or index level names to join on in the **left** DataFrame.

- `right_on`: Column or index level names to join on in the **right** DataFrame.

In [80]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [81]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [82]:
new_pd = pd.merge(df1, df3, left_on='employee', right_on='name')
new_pd

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


The result has a redundant column that we can drop if desired

In [83]:
new_pd.drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The `left_index` and `right_index` keywords

Merge on an index.

In [84]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [85]:
df1a = df1.set_index('employee')
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [86]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [87]:
df2a = df2.set_index('employee')
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


Use the index as the key for merging by specifying the `left_index` and/or `right_index` flags in `pd.merge()`:

In [88]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


`DataFrame` also has a `join()` method, which performs a merge that defaults to joining **on indices**

In [89]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


If you'd like to mix indices and columns, you can **combine `left_index` with `right_on` or `left_on` with `right_index`** to get the desired behavior:

In [90]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [91]:
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [92]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


In [93]:
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


## Specify Set Arithmetic for Joins

In [94]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [95]:
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [96]:
pd.merge(df6, df7)

,name,food,drink
0,Mary,bread,wine


Here we have merged two datasets that have only a single "name" entry in common: Mary. **By default, the result contains the intersection of the two sets of inputs; this is what is known as an *inner join*.**

We can specify this explicitly using the `how` keyword, which defaults to `"inner"`:

In [97]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


*Outer join*: returns a join over the union of the input columns, and fills in all missing values with NAs

In [98]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


*left join* and *right join*:  return joins over the left entries and right entries, respectively.

In [99]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


The output rows now correspond to the entries in the left input. 

In [100]:
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


## Overlapping Column Names: The `suffixes` Keyword

Two input `DataFrame`s have conflicting column names

In [101]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [102]:
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [103]:
pd.merge(df8, df9, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Because the output would have two conflicting column names (two "rank" columns), the merge function automatically appends a **suffix** `_x` or `_y` to make the output columns unique. 

If these defaults are inappropriate, it is possible to specify a custom suffix using the `suffixes` keyword:

In [104]:
pd.merge(df8, df9, on='name', suffixes=["_L", "_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


## Example: Us States Data 

Merge and join operations come up most often when combining data from different sources. 

### Read CSV

In [105]:
pop = pd.read_csv('../data/state-population.csv')
areas = pd.read_csv('../data/state-areas.csv')
abbrevs = pd.read_csv('../data/state-abbrevs.csv')

In [106]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [107]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [108]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


**Task: rank US states and territories by their 2010 population density.**

### Handle `state` column

Start with a many-to-one merge that will give us the full state name within the population `DataFrame`

Merge based on the `state/region` column of `pop`, and the `abbreviation` column of `abbrevs`.

In [109]:
merged = pd.merge(pop, abbrevs, how='outer', 
                  left_on='state/region', right_on='abbreviation')
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


Drop duplicate information: the `abbreviation` column

In [110]:
merged = merged.drop('abbreviation', axis=1)

In [111]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [112]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the `population` info is null:

In [113]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely **ue to this data not being available from the original source.**

More importantly, we see also that some of the new `state` entries are also null, which means that there was no corresponding entry in the `abbrevs` key! 

In [114]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

-> Address the issue:

Our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key. 

Fix this problem by filling in appropriate entries:

In [116]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United State'

In [118]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the `state` column!

### Merge with area data

In [119]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [120]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [122]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


### Check and Handle NAs in other columns

In [123]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Our `areas` `DataFrame` does not contain the area of the United States as a whole. We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

In [136]:
final.dropna(inplace=True)
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


In [138]:
# Double check for NAs
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

Now we have all the data we need. 

### Rank US states and territories by their 2010 population density

#### Select the portion of the data corresponding with the year 2000, and the total population

In [141]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


#### Compute the population denstiy and display it in order

Reindexing data on the state

In [142]:
data2010.set_index('state', inplace=True)

In [144]:
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


Compute the density

In [151]:
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)

In [152]:
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile. We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

In [153]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.